In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import trimesh

from magnet_pinn.generator.io import MeshWriter
from magnet_pinn.generator.phantoms import Tissue
from magnet_pinn.generator.properties import PropertySampler
from magnet_pinn.generator.transforms import ToMesh, MeshesCutout, MeshesCleaning, Compose

First generate 3d objects we suppose to use

In [3]:
tissue = Tissue(
    num_children_blobs=3,
    initial_blob_radius=100,
    initial_blob_center={
        "x": [-5, 5],
        "y": [-5, 5],
        "z": [-50, 50],
    },
    blob_radius_decrease_per_level=0.3,
    num_tubes=10,
    relative_tube_max_radius=0.1,
    relative_tube_min_radius=0.01
)
raw_3d_stgructures = tissue.generate()
raw_3d_stgructures

PhantomItem(parent=Blob(relative_disruption_strength=0.1, empirical_max_offset=np.float64(0.15813752314564616), empirical_min_offset=np.float64(-0.15047623745091002)), children=[Blob(relative_disruption_strength=0.1, empirical_max_offset=np.float64(0.1516295672123887), empirical_min_offset=np.float64(-0.1504481593837947)), Blob(relative_disruption_strength=0.1, empirical_max_offset=np.float64(0.14130894812371148), empirical_min_offset=np.float64(-0.14764688687832678)), Blob(relative_disruption_strength=0.1, empirical_max_offset=np.float64(0.16300497316937143), empirical_min_offset=np.float64(-0.17499048060748473))], tubes=[Tube(direction=array([ 0.85186786,  0.50989452, -0.11970269]), height=10000), Tube(direction=array([ 0.59301228,  0.32578067, -0.73634461]), height=10000), Tube(direction=array([-0.24478315, -0.33341478, -0.91044813]), height=10000), Tube(direction=array([-0.99672527, -0.0764441 ,  0.02636354]), height=10000), Tube(direction=array([-0.04957907,  0.62556775,  0.778592

Process 3d obejcts into the meshes

In [4]:
workflow = Compose([
    ToMesh(),
    MeshesCutout(),
    MeshesCleaning()
])
meshes = workflow(raw_3d_stgructures)
meshes

PhantomItem(parent=<trimesh.Trimesh(vertices.shape=(42474, 3), faces.shape=(84996, 3))>, children=[<trimesh.Trimesh(vertices.shape=(10377, 3), faces.shape=(20758, 3))>, <trimesh.Trimesh(vertices.shape=(10016, 3), faces.shape=(20028, 3))>, <trimesh.Trimesh(vertices.shape=(10167, 3), faces.shape=(20338, 3))>], tubes=[<trimesh.Trimesh(vertices.shape=(112, 3), faces.shape=(220, 3))>, <trimesh.Trimesh(vertices.shape=(114, 3), faces.shape=(224, 3))>, <trimesh.Trimesh(vertices.shape=(108, 3), faces.shape=(212, 3))>, <trimesh.Trimesh(vertices.shape=(156, 3), faces.shape=(308, 3))>, <trimesh.Trimesh(vertices.shape=(137, 3), faces.shape=(270, 3))>, <trimesh.Trimesh(vertices.shape=(184, 3), faces.shape=(364, 3))>, <trimesh.Trimesh(vertices.shape=(150, 3), faces.shape=(296, 3))>, <trimesh.Trimesh(vertices.shape=(258, 3), faces.shape=(512, 3))>, <trimesh.Trimesh(vertices.shape=(225, 3), faces.shape=(446, 3))>, <trimesh.Trimesh(vertices.shape=(171, 3), faces.shape=(338, 3))>])

In [5]:
trimesh.boolean.union(
    meshes.children + meshes.tubes,
    engine='manifold',
).show()

Sample properties for each mesh

In [6]:
prop_sampler = PropertySampler(
    {
        "density": {
            "min": 400,
            "max": 2000
        },
        "conductivity": {
            "min": 0,
            "max": 2.5
        },
        "permittivity": {
            "min": 1,
            "max": 71
        }
    }
)
prop = prop_sampler.sample_like(meshes)
prop

PhantomItem(parent=PropertyItem(conductivity=2.3463659403923876, permittivity=45.98474168254817, density=506.18899739505514), children=[PropertyItem(conductivity=1.9170459711857546, permittivity=8.508227887996714, density=1248.6578368795408), PropertyItem(conductivity=2.403939403902192, permittivity=12.73547849899547, density=935.1924293070722), PropertyItem(conductivity=0.6231395072407506, permittivity=48.81665748145046, density=613.1844175214324)], tubes=[PropertyItem(conductivity=0.9254486039119331, permittivity=6.202144562682283, density=1477.7122953918938), PropertyItem(conductivity=1.538090529283851, permittivity=50.096538907790475, density=820.5777370894459), PropertyItem(conductivity=1.2481398593788862, permittivity=65.09055214270386, density=1579.2218855488663), PropertyItem(conductivity=0.7000355954622848, permittivity=19.52117926530101, density=504.0617288339462), PropertyItem(conductivity=1.194480935535189, permittivity=50.46449108506982, density=1608.6424291268775), Proper

Now export meshes as `.stl` files and also pandas dataframe with columns `file`, `permittivity`, `conductivity`, `density` into the `materials.txt` file in the `.csv` format.

In [7]:
writer = MeshWriter("../gen_data/raw/tissue_meshes")
writer.write(meshes, prop)